## MOUNTING GOOGLE DRIVE

In [ ]:
using_colab = True
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

## INSTALL SEGMENT ANYTHING MODEL

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth

## INSTALL LIBRARIES

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

## CALL AND DISPLAY IMAGE

In [ ]:
image = cv2.imread('')# add your path here
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
dpi = 100
fig_width = 1024 / dpi  # 10.24 inches
fig_height = 691 / dpi  # 6.91 inches

plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
plt.imshow(image)
plt.axis('off')
plt.show()

## SETUP SAM

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_l_0b3195.pth"
model_type = "vit_l"

device = torch.device("cuda") ## change to cpu if no gpu available

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

## DISPLAYING IMAGES WITH THE SEGMENTED IMAGE MASKS

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
masks = mask_generator.generate(image)
print(len(masks))
print(masks[0].keys())
dpi = 100
fig_width = 1024 / dpi  # 10.24 inches
fig_height = 691 / dpi  # 6.91 inches

plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show()

In [ ]:
mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=52,
    pred_iou_thresh=0.7,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=60,  # Requires open-cv to run post-processing
)

masks2 = mask_generator_2.generate(image)
len(masks2)

dpi = 100
fig_width = 1024 / dpi  # 10.24 inches
fig_height = 691 / dpi  # 6.91 inches

plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
plt.imshow(image)
show_anns(masks2)
plt.axis('off')
plt.show()

## SETUP CLUSTERING AND APPLY CLUSTERING

In [ ]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def filter_masks_by_size(masks, image, max_size):
    """
    Filters masks based on their size to remove very small and very large masks.
    min_size and max_size should be defined based on the domain knowledge about the grain sizes.
    """
    filtered_masks = []
    total_pixels = image.shape[0] * image.shape[1]
    for mask in masks:
        mask_size = np.sum(mask['segmentation'])
        if mask_size <= max_size:
            filtered_masks.append(mask)
    return filtered_masks

def calculate_circularity(mask):
    area = np.sum(mask)
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        perimeter = cv2.arcLength(contours[0], True)
        if perimeter == 0:
            return 0
        circularity = 4 * np.pi * (area / (perimeter ** 2))
    else:
        circularity = 0
    return circularity
def average_intensity(image, mask):
    masked_image = cv2.bitwise_and(image, image, mask=mask.astype(np.uint8))
    return np.mean(masked_image[mask > 0])

def calculate_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou = np.sum(intersection) / np.sum(union)
    return iou

def apply_kmeans(masks, image):
    avg_intensities = [average_intensity(image, mask['segmentation']) for mask in masks]
    features = np.array(avg_intensities).reshape(-1, 1)
    kmeans = KMeans(n_clusters=2, random_state=0).fit(features)
    labels = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_
    darker_cluster_index = 0 if cluster_centers[0] < cluster_centers[1] else 1
    darker_masks = [masks[i] for i in range(len(masks)) if labels[i] == darker_cluster_index]
    lighter_masks = [masks[i] for i in range(len(masks)) if labels[i] != darker_cluster_index]
    return darker_masks,lighter_masks



def show_masks(image, masks,title="Masks"):
    plt.figure(figsize=(20, 20))
    plt.imshow(image)
    show_anns(masks)
    plt.title(title)
    plt.axis('off')
    plt.show()

def resolve_overlaps(masks):
    keep_masks = []
    n = len(masks)
    removed = set()

    for i in range(n):
        if i in removed:
            continue
        current_mask = masks[i]['segmentation']
        max_area = np.sum(current_mask)
        best_mask = masks[i]

        for j in range(i + 1, n):
            if j in removed:
                continue
            compare_mask = masks[j]['segmentation']
            if calculate_iou(current_mask, compare_mask) > 0.5:  # Threshold for considering an overlap
                if np.sum(compare_mask) > max_area:
                    max_area = np.sum(compare_mask)
                    removed.add(i)
                    best_mask = masks[j]
                    current_mask = compare_mask
                else:
                    removed.add(j)

        keep_masks.append(best_mask)

    return keep_masks





# Assuming masks and image are defined
darker_masks1, lighter_masks1 = apply_kmeans(masks, image)
darker_masks2, lighter_masks2 = apply_kmeans(masks2, image)
# Combine and resolve overlaps for darker masks
combined_darker_masks =  darker_masks2
non_overlapping_darker = resolve_overlaps(combined_darker_masks)
show_masks(image, combined_darker_masks, "Darker Masks")

# Calculate and display properties for lighter masks
combined_lighter_masks =  lighter_masks2
non_overlapping_lighter = resolve_overlaps(combined_lighter_masks)
max_size = 0.1 * image.shape[0] * image.shape[1]   # Example: 10% of the image area
filtered_lighter_masks = filter_masks_by_size(combined_lighter_masks, image, max_size)
show_masks(image, filtered_lighter_masks, "Lighter Masks")



## EXTRACT GRAIN AREAS AND DISPLAY DISTRIBUTION GRAPH

In [ ]:
import numpy as np
import pandas as pd
import altair as alt

def extract_and_print_areas(masks, pixel_to_um_scale):
    """
    Extracts areas from masks, converts them to µm², and prints them.
    """
    areas_pixels = [mask['area'] for mask in masks]
    areas_um2 = [area * pixel_to_um_scale for area in areas_pixels]
    average_area_pixels = np.mean(areas_pixels) if areas_pixels else 0
    average_area_um2 = np.mean(areas_um2) if areas_um2 else 0

    print("Number of masks:", len(masks))
    print("Areas of all masks in pixels:", areas_pixels)
    print("Areas of all masks in µm²:", areas_um2)
    print(f"Average area of masks: {average_area_pixels:.2f} pixels")
    print(f"Average area of masks: {average_area_um2:.2f} µm²")

    return areas_um2  # Return areas in µm² for further processing

def remove_outliers(data, lower_percentile=25, upper_percentile=75, factor=1.5):
    """
    Removes outliers from data based on the IQR method.
    """
    q1 = np.percentile(data, lower_percentile)
    q3 = np.percentile(data, upper_percentile)
    iqr = q3 - q1
    lower_bound = q1 - (factor * iqr)
    upper_bound = q3 + (factor * iqr)

    return [x for x in data if lower_bound <= x <= upper_bound]

def plot_distribution_areas(areas_um2):
    """
    Plots the distribution of areas without outliers using Altair.
    """
    # Remove outliers
    areas_um2_filtered = remove_outliers(areas_um2)

    # Convert the filtered areas to a DataFrame
    data = pd.DataFrame({'Area_um2': areas_um2_filtered})

    # Create the distribution bar chart without normalization and without outliers
    bar_chart = alt.Chart(data).mark_bar().encode(
        alt.X('Area_um2:Q', bin=alt.Bin(maxbins=100), axis=alt.Axis(labelAngle=90, format='.3f')),  # Set more significant digits and vertical labels
        alt.Y('count()'),
    ).properties(
        width=500,
        height=300,
        title='Distribution of Grain Areas'
    ).configure_axis(
        labelFont='Arial',
        labelFontSize=7,
        titleFont='Arial',
        titleFontSize=7
    ).configure_title(
        font='Arial',
        fontSize=7
    )

    return bar_chart

# Assuming your `masks` and `filtered_lighter_masks` are already defined from earlier in the code.
# Scale factor for converting pixel area to square micrometers (µm²)
scale = ## please add the conversion scale
pixel_to_um_scale = (scale)** 2

# Extract and print areas for lighter masks
print("Lighter Masks:")
areas_lighter_um2 = extract_and_print_areas(filtered_lighter_masks, pixel_to_um_scale)

# Plot distribution for lighter masks without normalization and with outliers removed
chart_lighter = plot_distribution_areas(areas_lighter_um2)
chart_lighter.display()


## GRAIN EXTRACTION AND DISTRIBUTION CHART FOR NON DEFECT SAMPLES

In [ ]:
def filter_masks_by_size(masks, image, max_size):
    """
    Filters masks based on their size to remove very small and very large masks.
    """
    filtered_masks = []
    total_pixels = image.shape[0] * image.shape[1]
    for mask in masks:
        mask_size = np.sum(mask['segmentation'])
        if mask_size <= max_size:
            filtered_masks.append(mask)
    return filtered_masks

def calculate_circularity(mask):
    area = np.sum(mask)
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        perimeter = cv2.arcLength(contours[0], True)
        if perimeter == 0:
            return 0
        circularity = 4 * np.pi * (area / (perimeter ** 2))
    else:
        circularity = 0
    return circularity

def average_intensity(image, mask):
    masked_image = cv2.bitwise_and(image, image, mask=mask.astype(np.uint8))
    return np.mean(masked_image[mask > 0])

def calculate_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou = np.sum(intersection) / np.sum(union)
    return iou

def show_masks(image, masks, title="Masks"):
    plt.figure(figsize=(20, 20))
    plt.imshow(image)
    show_anns(masks)
    plt.title(title)
    plt.axis('off')
    plt.show()

def resolve_overlaps(masks):
    keep_masks = []
    n = len(masks)
    removed = set()

    for i in range(n):
        if i in removed:
            continue
        current_mask = masks[i]['segmentation']
        max_area = np.sum(current_mask)
        best_mask = masks[i]

        for j in range(i + 1, n):
            if j in removed:
                continue
            compare_mask = masks[j]['segmentation']
            if calculate_iou(current_mask, compare_mask) > 0.5:  # Threshold for considering an overlap
                if np.sum(compare_mask) > max_area:
                    max_area = np.sum(compare_mask)
                    removed.add(i)
                    best_mask = masks[j]
                    current_mask = compare_mask
                else:
                    removed.add(j)

        keep_masks.append(best_mask)

    return keep_masks

# Assuming masks and masks2 are defined
# Merge masks from both sources
combined_masks =  masks2

# Resolve overlaps for combined masks
#non_overlapping_masks = resolve_overlaps(combined_masks)

# Show resolved masks
show_masks(image, combined_masks, "Resolved Masks")

# Filter lighter masks by size (optional, based on domain knowledge)
max_size = 0.1 * image.shape[0] * image.shape[1]   # Example: 10% of the image area
filtered_masks = filter_masks_by_size(combined_masks, image, max_size)

# Display the filtered masks
show_masks(image, filtered_masks, "Filtered Masks")

In [ ]:
import pandas as pd
import altair as alt
def extract_and_print_areas(masks, pixel_to_um_scale):
    """
    Extracts areas from masks, converts them to µm², and prints them.
    """
    areas_pixels = [mask['area'] for mask in masks]
    areas_um2 = [area * pixel_to_um_scale for area in areas_pixels]
    average_area_pixels = np.mean(areas_pixels) if areas_pixels else 0
    average_area_um2 = np.mean(areas_um2) if areas_um2 else 0

    print("Number of masks:", len(masks))
    print("Areas of all masks in pixels:", areas_pixels)
    print("Areas of all masks in µm²:", areas_um2)
    print(f"Average area of masks: {average_area_pixels:.2f} pixels")
    print(f"Average area of masks: {average_area_um2:.2f} µm²")

    return areas_um2  # Return areas in µm² for further processing

def remove_outliers(data, lower_percentile=25, upper_percentile=75, factor=1.5):
    """
    Removes outliers from data based on the IQR method.
    """
    q1 = np.percentile(data, lower_percentile)
    q3 = np.percentile(data, upper_percentile)
    iqr = q3 - q1
    lower_bound = q1 - (factor * iqr)
    upper_bound = q3 + (factor * iqr)

    return [x for x in data if lower_bound <= x <= upper_bound]

def plot_distribution_areas(areas_um2):
    """
    Plots the distribution of areas without outliers using Altair.
    """
    # Remove outliers
    areas_um2_filtered = remove_outliers(areas_um2)

    # Convert the filtered areas to a DataFrame
    data = pd.DataFrame({'Area_um2': areas_um2_filtered})

    # Create the distribution bar chart without normalization and without outliers
    bar_chart = alt.Chart(data).mark_bar().encode(
        alt.X('Area_um2:Q', bin=alt.Bin(maxbins=100), axis=alt.Axis(labelAngle=90, format='.3f')),  # Set more significant digits and vertical labels
        alt.Y('count()'),
    ).properties(
        width=500,
        height=300,
        title='Distribution of Grain Areas'
    ).configure_axis(
        labelFont='Arial',
        labelFontSize=7,
        titleFont='Arial',
        titleFontSize=7
    ).configure_title(
        font='Arial',
        fontSize=7
    )

    return bar_chart

# Assuming your `filtered_masks` are already defined from earlier in the code.
# Scale factor for converting pixel area to square micrometers (µm²)
scale = ## please add the conversion scale
pixel_to_um_scale = scale** 2

# Extract and print areas for filtered masks
print("Filtered Masks:")
areas_filtered_um2 = extract_and_print_areas(filtered_masks, pixel_to_um_scale)

# Plot distribution for filtered masks without normalization and with outliers removed
chart_filtered = plot_distribution_areas(areas_filtered_um2)
chart_filtered.display()




## SEGMENT AND DISPLAY MASK IN SAME COLOR

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

def show_anns(anns, color):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([color, [0.35]])  # Use the provided color
        img[m] = color_mask
    ax.imshow(img)

# image = cv2.imread('/content/drive/MyDrive/HOLE_Detection/Antisolvent_none_2_8.jpg')
image = cv2.imread('/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/NO HOLES/Temp 50-150_150c1.tif')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

dpi = 100
fig_width = 1024 / dpi  # 10.24 inches
fig_height = 691 / dpi  # 6.91 inches

plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
plt.imshow(image)
plt.axis('off')
plt.show()

import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_l_0b3195.pth"
model_type = "vit_l"

device = torch.device("cuda")

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

masks = mask_generator.generate(image)

# Define the color for masks (e.g., red)
mask_color = [1, 0, 0]  # Red color (R, G, B)
dpi = 100
fig_width = 1024 / dpi  # 10.24 inches
fig_height = 691 / dpi  # 6.91 inches

plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
plt.imshow(image)
show_anns(masks, mask_color)
plt.axis('off')
plt.show()

mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=52,
    pred_iou_thresh=0.7,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=60,  # Requires open-cv to run post-processing
)

# Generate second set of masks (masks2)
masks2 = mask_generator_2.generate(image)

# Define the color for masks2 (e.g., blue)
masks2_color = [0, 0, 1]  # Blue color (R, G, B)

dpi = 100
fig_width = 1024 / dpi  # 10.24 inches
fig_height = 691 / dpi  # 6.91 inches

plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
plt.imshow(image)
show_anns(masks2, masks2_color)  # Use masks2_color for the second set of masks
plt.axis('off')
plt.show()
